In [2]:
import os
import glob
import json
import numpy as np
import mplhep as hep
import awkward as ak
import matplotlib.pyplot as plt

hep.style.use(hep.styles.CMS)

In [25]:
FRACS = ["2e3", "1e4", "1e5", "1e6"]
MODEL_TYPES = {
    'fine_tuning': "Fine-tuning",
    'from_scratch': "From scratch",
    'fixed_backbone': "Fine-tuning \n(fixed backbone)",
    
}
TRAINING_TYPES = ['jet_regression', 'dm_multiclass', "binary_classification"]
PERFORMANCE_KEYS = {
    "jet_regression": ["loss_validation", "IQR_validation", "median_validation"],
    "binary_classification": ["loss_validation", "precision_validation", "recall_validation", "F1_validation"],
    "dm_multiclass": ["loss_validation", "precision_validation", "recall_validation", "F1_validation"]
}
Y_LABELS = {
    "IQR_validation": r"$p_T \, \, resol. (q_{75} - q_{25})/q_{50}$",
    "median_validation": r"$p_T scale (q_{50})$",
    "precision_validation": "precision",
    "recall_validation": "recall",
    "F1_validation": "F1",
    "loss_validation": "validation loss"
}

COLORS = {
    "fine_tuning": "green",
    "from_scratch": "red",
    "fixed_backbone": "blue",
}
EPOCH_COLORS = {
    "epoch_0": "green",
    "epoch_30": "red",
    "epoch_60": "magenta",
    "fixed_backbone": "blue",
}
LAYER_COLORS = {
    "GPT_layers_1": "green",
    "GPT_layers_2": "red",
    "GPT_layers_3": "blue",
}


EPOCH_0_UNFREEZE_DIR = "/home/laurits/tmp/20250117_unfreeze_epoch_0/"
EPOCH_60_UNFREEZE_DIR = "/home/laurits/tmp/20250117_unfreeze_epoch_60/"
BASE_DIR = "/home/laurits/tmp/20241204_ParT_ntrain_v2"
GPT_LAYERS_1_DIR = "/home/laurits/tmp/20250119_GPT_Layers_1/"
GPT_LAYERS_2_DIR = "/home/laurits/tmp/20250119_GPT_Layers_2/"
OUTPUT_DIR = "/home/laurits/tmp/20250119_OmniParT_effects"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# Effects of unfreezing

In [16]:
epoch_dirs = {
    "epoch_0": EPOCH_0_UNFREEZE_DIR,
    "epoch_30": BASE_DIR,
    "epoch_60": EPOCH_60_UNFREEZE_DIR,
    "fixed_backbone": BASE_DIR
}

epoch_name_mapping = {
    "epoch_0": "Epoch 0",
    "epoch_30": "Epoch 30",
    "epoch_60": "Epoch 60",
    "fixed_backbone": "Frozen"
}


def calculate_scenario_measures(wcp_path, keys):
    model_metrics = {key: {} for key in keys}
    metric_values = {key: [] for key in keys}
    for metrics_path in glob.glob(wcp_path):  # Loop over all runs (v_1, .. v_n)
        if not os.path.exists(metrics_path):
            print(f"ERROR: {metrics_path} is not started")
            continue
        with open(metrics_path, "rt") as in_file:
            metrics_info = json.load(in_file)
        min_loss_idx = np.argmin(metrics_info["loss_validation"])
        for key in keys:
            metric_values[key].append(np.nan_to_num(metrics_info[key][min_loss_idx]))
    for key in keys:
        model_metrics[key]["mean"] = np.mean(metric_values[key])
        model_metrics[key]["std"] = np.std(metric_values[key])
    return model_metrics


def restructure_results(model_metrics: dict, keys):
    restructured_results = {key: {} for key in keys}
    for key in keys:
        means = []
        stdevs = []
        for frac in FRACS:
            means.append(model_metrics[frac][key]["mean"])
            stdevs.append(model_metrics[frac][key]["std"])
        restructured_results[key]["mean"] = means
        restructured_results[key]["std"] = stdevs
    return restructured_results        
    


def get_model_metrics_values(training_type, em):
    model = "OmniParT_fine_tuning" if "epoch" in em else "OmniParT_fixed_backbone"
    keys = PERFORMANCE_KEYS[training_type]
    model_metrics = {}
    for frac in FRACS:
        wcp_path = os.path.join(epoch_dirs[em], "*", f"trainfrac_{frac}", training_type, model, "history.json")
        model_metrics[frac] = calculate_scenario_measures(wcp_path, keys)
    restructured_results = restructure_results(model_metrics, keys)
    return restructured_results


def get_unfreezing_metrics_values(training_type):
    epoch_models = epoch_name_mapping.keys()
    model_versions = {}
    for em in epoch_models:
        model_versions[em] = get_model_metrics_values(training_type, em)
    return model_versions


def plot_performance(metrics_values, key, output_path):
    fs = [float(f) for f in FRACS]
    for m_type, name in epoch_name_mapping.items():
        mean = np.array(metrics_values[m_type][key]["mean"])
        std = np.array(metrics_values[m_type][key]["std"])
        plt.plot(fs, mean, label=name, color=EPOCH_COLORS[m_type])
        plt.fill_between(fs, mean - std, mean + std, color=EPOCH_COLORS[m_type], alpha=0.3)
    plt.legend()
    plt.xscale("log")
    plt.xlabel("Number of training jets")
    plt.ylabel(Y_LABELS[key])
    plt.savefig(output_path, bbox_inches="tight", format="pdf")
    plt.close("all")


def plot_all_performances(training_type):
    metrics_values = get_unfreezing_metrics_values(training_type)
    output_dir = os.path.join(OUTPUT_DIR, training_type)
    os.makedirs(output_dir, exist_ok=True)
    for key in PERFORMANCE_KEYS[training_type]:
        output_path = os.path.join(output_dir, f"{key}_layerFreeze.pdf")
        plot_performance(metrics_values, key, output_path)

In [11]:
for training_type in PERFORMANCE_KEYS.keys():
    plot_all_performances(training_type)

# Effects of number of GPT layers

In [23]:
layer_dirs = {
    "GPT_layers_1": GPT_LAYERS_1_DIR,
    "GPT_layers_2": GPT_LAYERS_2_DIR,
    "GPT_layers_3": BASE_DIR,
}

layer_name_mapping = {
    "GPT_layers_1": "1 GPT layer",
    "GPT_layers_2": "2 GPT layers",
    "GPT_layers_3": "3 GPT layers",
}


def get_model_metrics_values_gptLayer(training_type, em):
    keys = PERFORMANCE_KEYS[training_type]
    model_metrics = {}
    for frac in FRACS:
        wcp_path = os.path.join(layer_dirs[em], "*", f"trainfrac_{frac}", training_type, "OmniParT_fine_tuning", "history.json")
        model_metrics[frac] = calculate_scenario_measures(wcp_path, keys)
    restructured_results = restructure_results(model_metrics, keys)
    return restructured_results


def get_gptLayer_metrics_values(training_type):
    model_types = layer_name_mapping.keys()
    model_versions = {}
    for em in model_types:
        model_versions[em] = get_model_metrics_values_gptLayer(training_type, em)
    return model_versions


def plot_performance_gptLayer(metrics_values, key, output_path):
    fs = [float(f) for f in FRACS]
    for m_type, name in layer_name_mapping.items():
        mean = np.array(metrics_values[m_type][key]["mean"])
        std = np.array(metrics_values[m_type][key]["std"])
        plt.plot(fs, mean, label=name, color=LAYER_COLORS[m_type])
        plt.fill_between(fs, mean - std, mean + std, color=LAYER_COLORS[m_type], alpha=0.3)
    plt.legend()
    plt.xscale("log")
    plt.xlabel("Number of training jets")
    plt.ylabel(Y_LABELS[key])
    plt.savefig(output_path, bbox_inches="tight", format="pdf")
    plt.close("all")


def plot_all_performances_gptLayer(training_type):
    metrics_values = get_gptLayer_metrics_values(training_type)
    output_dir = os.path.join(OUTPUT_DIR, training_type)
    os.makedirs(output_dir, exist_ok=True)
    for key in PERFORMANCE_KEYS[training_type]:
        output_path = os.path.join(output_dir, f"{key}_gptLayerCount.pdf")
        plot_performance_gptLayer(metrics_values, key, output_path)

In [24]:
for training_type in PERFORMANCE_KEYS.keys():
    plot_all_performances_gptLayer(training_type)